<a href="https://colab.research.google.com/github/syt73/python/blob/main/pythonbasic12_Machine_Learning_%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%892.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [203]:
# G 드라이브 연결
from google.colab import drive
drive.mount('/gdrive/')

# 모듈 설정
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns


# https://dacon.io/competitions/open/235539/data

train = pd.read_csv('/gdrive/My Drive/Colab Notebooks/titanic/train.csv')
test = pd.read_csv('/gdrive/My Drive/Colab Notebooks/titanic/test.csv')
submission = pd.read_csv('/gdrive/My Drive/Colab Notebooks/titanic/submission.csv')


# 데이터 결합 (위 아래 방향으로 결합)

# 분석을 끝내고 학습을 시킬때 다시 분리 하기 편하도록 split  열을 만들어 각각 이름을 부여한다. 
train['Split'] = 'Train'
test['Split'] = 'Test'

data = pd.concat([train, test], axis=0)

data_num = data.loc[:,['Pclass','Age','SibSp','Parch','Fare','Survived']] 

# age  평균
data_num['Age'] = data_num['Age'].fillna(data_num['Age'].mean())

# fare 최빈
data_num['Fare'] = data_num['Fare'].fillna(data_num['Fare'].mode()[0])



Drive already mounted at /gdrive/; to attempt to forcibly remount, call drive.mount("/gdrive/", force_remount=True).


In [204]:
# 피처 엔지니어링 - 각 피처 열을 모델 학습에 맞게 전처리하고 data['Sex']
data['Sex']

0        male
1      female
2      female
3      female
4        male
        ...  
413      male
414    female
415      male
416      male
417      male
Name: Sex, Length: 1309, dtype: object

In [205]:
# female 0  male 1

data.loc[data['Sex']=='female','Sex'] = 0
data.loc[data['Sex']=='male','Sex'] = 1

data['Sex']

0      1
1      0
2      0
3      0
4      1
      ..
413    1
414    0
415    1
416    1
417    1
Name: Sex, Length: 1309, dtype: object

In [206]:
data['Sex'] = data['Sex'].astype(int)
data['Sex']

0      1
1      0
2      0
3      0
4      1
      ..
413    1
414    0
415    1
416    1
417    1
Name: Sex, Length: 1309, dtype: int64

In [207]:
# Name
data['Name']

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
413                                   Spector, Mr. Woolf
414                         Oliva y Ocana, Dona. Fermina
415                         Saether, Mr. Simon Sivertsen
416                                  Ware, Mr. Frederick
417                             Peter, Master. Michael J
Name: Name, Length: 1309, dtype: object

In [208]:
title_name = data['Name'].str.split(", ", expand=True)[1]

# 띄어 쓰기 주의...점 뒤에 한칸 있음

In [209]:
   # 시리즈 데이터형을 str로 변환하여 자르고 인덱싱
title = title_name.str.split(".", expand=True)[0]
title.value_counts(dropna=False)

Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Mlle              2
Major             2
Ms                2
Lady              1
Sir               1
Mme               1
Don               1
Capt              1
the Countess      1
Jonkheer          1
Dona              1
Name: 0, dtype: int64

In [210]:
# 비슷한 속성을 가진 타이틀 들을 그룹으로 묶자. 

title = title.replace(['Ms'], 'Miss')
title = title.replace(['Mlle', 'the Countess', 'Lady', 'Don', 'Dona', 'Mme', 'Sir', 'Jonkheer'], 'Noble')
title = title.replace(['Col', 'Major', 'Capt'], 'Officer')
title = title.replace(['Dr', 'Rev'], 'Priest')
data['Title'] = np.array(title)
data['Title'].value_counts(dropna=False)

Mr         757
Miss       262
Mrs        197
Master      61
Priest      16
Noble        9
Officer      7
Name: Title, dtype: int64

In [211]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Split,Title
0,1,0.0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S,Train,Mr
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C,Train,Mrs
2,3,1.0,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Train,Miss
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S,Train,Mrs
4,5,0.0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S,Train,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",1,NaN,0,0,A.5. 3236,8.0500,NaN,S,Test,Mr
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,108.9000,C105,C,Test,Noble
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Test,Mr
416,1308,NaN,3,"Ware, Mr. Frederick",1,NaN,0,0,359309,8.0500,NaN,S,Test,Mr


In [212]:
# Title 열을 만들었으므로 필요 없어진 name 열은 삭제한다. 

# data= data.drop('Name', axis=1)
# data

In [213]:
# 나이대

bins = [0, 4, 8, 12, 16, 32, 36, 48, 56, 64, 100]
labels = ['Infant','Child1','Chind2','Youth1','Youth2','Adult1','Adult2','Middle Aged','Senior','Elderly']

data['AgeBin'] = pd.cut(data['Age'], bins=bins, labels=labels)

In [214]:
# 형제자매 & 부모자식 열을 하나로 구성 하면서 본인까지 추가

data['FamilySize'] = data['SibSp'] + data['Parch'] + 1

In [215]:
# 요금에 1개의 결측치 처리

data.loc[data['Fare'].isnull(),:]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Split,Title,AgeBin,FamilySize
152,1044,NaN,3,"Storey, Mr. Thomas",1,60.5,0,0,3701,NaN,NaN,S,Test,Mr,Senior,1


In [216]:
# 요금과 객실 등급의 상관 관계는 아주 높기 때문에 3등석 객실 승객의 탑승 요금의 평균값을 이용하여
# 결측치를 채운다. 
# 중요한 점은 전체 요금의 평균값이 아닌 3등급만의 평균을 계산하여 채운다. 

p3_fare_mean = data.loc[data['Pclass']==3,'Fare'].mean()

data['Fare'] = data['Fare'].fillna(p3_fare_mean)

data.loc[data['Fare'].isnull(),:]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Split,Title,AgeBin,FamilySize


In [217]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Split,Title,AgeBin,FamilySize
0,1,0.0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S,Train,Mr,Youth2,2
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C,Train,Mrs,Adult2,2
2,3,1.0,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Train,Miss,Youth2,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S,Train,Mrs,Adult1,2
4,5,0.0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S,Train,Mr,Adult1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",1,NaN,0,0,A.5. 3236,8.0500,NaN,S,Test,Mr,NaN,1
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,108.9000,C105,C,Test,Noble,Adult2,1
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Test,Mr,Adult2,1
416,1308,NaN,3,"Ware, Mr. Frederick",1,NaN,0,0,359309,8.0500,NaN,S,Test,Mr,NaN,1


In [218]:
# 레이블 인코딩 label Encoding

# 문자열 데이터로 이루어진 Title, AgeBin 을 숫자로 레이블로 바꿔주어야만 머신 러닝을 돌릴 수 있다. 

from sklearn.preprocessing import LabelEncoder

for col in ['Title','AgeBin']:
  encoder = LabelEncoder()
  data[col] = encoder.fit_transform(data[col])

data.loc[:,['Title','AgeBin']]

# 각 열에 속하는 범주의 개수만큼 숫자레이블로 변환해 준다. 

,Title,AgeBin
0,2,9
1,3,1
2,1,9
3,3,0
4,2,0
...,...,...
413,2,10
414,4,1
415,2,1
416,2,10


In [219]:
# 원핫 인코딩
# 문자열 데이터를 숫자형 데이터로 변경하는 또 다른 방법 

# onehot_prefix = []
# for col in ['Title', 'AgeBin']:
#   data[col] = data[col].astype('category')    # 그냥 문자열 데이터가 아닌 카테고리로 형변환 
#   data = pd.get_dummies(data, columns = [col], prefix = col[:3], drop_first= True)
#   onehot_prefix.append(col[:3])

# data.loc[:, [col for col in data.columns if col[:3] in onehot_prefix]].head()


In [220]:
# 피처 스케일링 - 모델 학습에 사용할 피처의 스케일을 0 ~ 1 범위로 정규화 처리한다. 

data_num = data.loc[:,['PassengerId','Survived','Pclass','Sex','Fare','Split','Title','AgeBin','FamilySize']]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaled_cols = [col for col in data_num.loc[:, 'Pclass':]. columns if col != 'Split']
data_scaled = data_num.loc[:, scaled_cols]
data_scaled = scaler.fit_transform(data_scaled)

data_scaled

array([[1.        , 1.        , 0.01415106, 0.33333333, 0.9       ,
        0.1       ],
       [0.        , 0.        , 0.13913574, 0.5       , 0.1       ,
        0.1       ],
       [1.        , 0.        , 0.01546857, 0.16666667, 0.9       ,
        0.        ],
       ...,
       [1.        , 1.        , 0.01415106, 0.33333333, 0.1       ,
        0.        ],
       [1.        , 1.        , 0.01571255, 0.33333333, 1.        ,
        0.        ],
       [1.        , 1.        , 0.0436405 , 0.        , 1.        ,
        0.2       ]])

In [221]:
data_num.loc[:, scaled_cols] = data_scaled[:,:]
data_num

,PassengerId,Survived,Pclass,Sex,Fare,Split,Title,AgeBin,FamilySize
0,1,0.0,1.0,1.0,0.014151,Train,0.333333,0.9,0.1
1,2,1.0,0.0,0.0,0.139136,Train,0.500000,0.1,0.1
2,3,1.0,1.0,0.0,0.015469,Train,0.166667,0.9,0.0
3,4,1.0,0.0,0.0,0.103644,Train,0.500000,0.0,0.1
4,5,0.0,1.0,1.0,0.015713,Train,0.333333,0.0,0.0
...,...,...,...,...,...,...,...,...,...
413,1305,NaN,1.0,1.0,0.015713,Test,0.333333,1.0,0.0
414,1306,NaN,0.0,0.0,0.212559,Test,0.666667,0.1,0.0
415,1307,NaN,1.0,1.0,0.014151,Test,0.333333,0.1,0.0
416,1308,NaN,1.0,1.0,0.015713,Test,0.333333,1.0,0.0


In [222]:
selected_features = ['Pclass','Sex','Fare','Title','AgeBin','FamilySize']

y_train = data_num.loc[data_num['Split']=='Train', 'Survived']
X_train = data_num.loc[data_num['Split']=='Train', selected_features]
X_test = data_num.loc[data_num['Split']=='Test', selected_features]  # 제출용 문제지

print('Train 데이터 셋 크기 : ', X_train.shape, y_train.shape)
print('Test 데이터 셋 크기 : ', X_test.shape)

Train 데이터 셋 크기 :  (891, 6) (891,)
Test 데이터 셋 크기 :  (418, 6)


In [223]:
# 훈련용과 검증용 8:2 로 분할

from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, random_state=20)


In [224]:
# 원하는 알고리즘으로 돌려서 코드와 정확도 결과 제출 


# 원하는 알고리즘으로 돌려서 정확도 결과 제출 

# 장서영   0.8770949720670391

from sklearn.tree import DecisionTreeClassifier                
dtc = DecisionTreeClassifier(max_depth=3, random_state = 20)   
dtc.fit(X_tr, y_tr)                                      

# 예측 (결과)

dtc_pred = dtc.predict(X_val)   
dtc_pred                       

# 평가
from sklearn.metrics import accuracy_score           
accuracy_score(y_val, dtc_pred)

0.8770949720670391

In [226]:
# 제출용 파일 만들기

y_test_pred = dtc.predict(X_test)
submission['Survived'] = y_test_pred.astype(int)

submission.to_csv('/gdrive/My Drive/Colab Notebooks/titanic/submission_007.csv', index=False)